In [163]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from d2d import *
from compile_fit import *


filename = "/data/fast0/datasets/Rhone_data_continuous.h5"
f = h5py.File(filename, 'r')
print("Keys: %s" % f.keys())

Keys: <KeysViewHDF5 ['DAS Data', 'Discharge', 'Times']>


In [164]:
# # Put data in a data frame
# das = np.transpose([chan[750] for chan in f['DAS Data']])

# # Might want to try this later:
# # das = np.log10(np.transpose([chan[750] for chan in f['DAS Data']]))

# discharge = np.array(f['Discharge'])
# df = pd.DataFrame(data={'das':das,'discharge':discharge})

In [165]:
das_data_all = f['DAS Data'][:]
discharge = f['Discharge'][:]

df_all_chan = pd.DataFrame(das_data_all)
df_all_chan['Discharge'] = discharge
df_all_chan.head()

,0,1,2,3,4,5,6,7,8,9,...,2487,2488,2489,2490,2491,2492,2493,2494,2495,Discharge
0,1737.318436,1932.321258,1788.269101,3706.977556,1831.756626,1768.674227,1908.850695,3690.341997,1804.145150,2437.436803,...,3603.932847,2066.070800,2215.669390,1881.772352,3990.883595,1584.965970,1787.281257,1862.912440,4160.545583,9.947179
1,8161.349279,8245.783723,8118.189935,528.834231,468.890511,714.714310,728.051775,158.440806,82.719811,116.949415,...,8238.452283,8204.901437,8194.671705,8228.769841,8211.066134,8229.370787,8218.436955,8210.143402,8178.032001,9.934657
2,8242.942236,8205.640252,8093.980026,510.832905,451.884623,709.510655,709.932307,153.715844,83.404382,116.112236,...,8184.500407,8221.490571,8202.429647,8194.927561,8197.675324,8188.718827,8189.658071,8225.017259,8163.918322,9.929290
3,8159.780047,8196.654224,8113.873839,489.734947,438.657688,690.418208,701.896139,158.058233,83.003254,112.787627,...,8173.860868,8156.187127,8151.113841,8235.843141,8145.360321,8192.392266,8169.275389,8129.926491,8161.652382,9.923923
4,8198.498465,8161.470921,8120.452338,521.425978,444.272741,689.710217,695.164917,160.246640,84.054029,111.125592,...,8198.827286,8234.815465,8219.987333,8140.961605,8193.517740,8157.154485,8241.648596,8229.194191,8234.212679,9.918557


In [166]:
# Plot the raw data

# fig,ax=plt.subplots(figsize=(15,5))
# ax.plot(das)
# ax.set_ylabel('DAS-measured strain rate')
# ax2=plt.twinx()
# ax2.plot(discharge,'-r')
# ax2.set_ylabel('Discharge')

In [167]:
# Split the data
column_indices = {name: i for i, name in enumerate(df_all_chan.columns)}
n = len(df_all_chan)

train_df = df_all_chan[int(n*0.3):int(n*1.0)]
val_df = df_all_chan[int(n*0.0):int(n*0.2)]
test_df = df_all_chan[int(n*0.2):int(n*0.3)]

In [168]:
input_columns = list(np.arange(0,2308,1))

In [169]:
# Normalize
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [170]:
# # Visualize the normalized data
# df_std = (df_all_chan - train_mean) / train_std
# df_std = df_std.melt(var_name='Column', value_name='Normalized')
# plt.figure(figsize=(12, 6))
# ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
# _ = ax.set_xticklabels(df.keys(), rotation=90)

# Does not like to visulaize with all the channels applied

In [171]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=0,
    train_df=train_df, 
    val_df=val_df, 
    test_df=test_df,
    label_columns=['Discharge'],
    input_columns=input_columns)

# Defining the models

In [172]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(1)
])

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

dnn_model = tf.keras.models.Sequential([
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
])

In [173]:
history = compile_and_fit(linear, single_step_window)

val_performance={}
performance={}
val_performance['Linear'] = linear.evaluate(single_step_window.val)
performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)

Epoch 1/20
1373/1373 [==============================] - 9s 6ms/step - loss: 0.4572 - mean_absolute_error: 0.4695 - val_loss: 1.2574 - val_mean_absolute_error: 0.9793
Epoch 2/20
1373/1373 [==============================] - 9s 6ms/step - loss: 0.4100 - mean_absolute_error: 0.4076 - val_loss: 3.5047 - val_mean_absolute_error: 1.5817
Epoch 3/20
1373/1373 [==============================] - 9s 6ms/step - loss: 0.2951 - mean_absolute_error: 0.3636 - val_loss: 0.9551 - val_mean_absolute_error: 0.8363
Epoch 4/20
1373/1373 [==============================] - 9s 7ms/step - loss: 0.2770 - mean_absolute_error: 0.3485 - val_loss: 1.4589 - val_mean_absolute_error: 1.0438
Epoch 5/20
393/393 [==============================] - 7s 521us/step - loss: 0.9569 - mean_absolute_error: 0.8395


In [174]:
# inputs, labels = single_step_window.example

# predictions = np.array(linear(inputs)).flatten()
# inputs = np.array(inputs).flatten()
# labels = np.array(labels).flatten()

# plt.subplots()
# plt.plot(labels,predictions,'ok')
# plt.plot((-1,1),(-1,1))
# plt.xlabel('Actual Normalized Discharge')
# plt.ylabel('Predicted Normalized Discharge')

# plt.subplots()
# plt.plot(inputs, labels,'ok')
# # for xx, yy in single_step_window.val.take(5):
# #     plt.plot(np.array(xx[:,0,:]).flatten(),np.array(yy).flatten(),'or')
# plt.xlabel('Actual Normalized DAS')
# plt.ylabel('Normalized Discharge (Actual and Modeled)')
# weights=linear.layers[0].kernel[:,0].numpy()
# biases=linear.layers[0].bias.numpy()
# plt.plot((-1,3.0),weights[0]*np.array((-1,3.0))+biases[0])

# # Compare to linear least squares
# from scipy import stats
# slope, intercept, r_value, p_value, std_err = stats.linregress(inputs,labels)
# plt.plot((-1,3.0),slope*np.array((-1,3.0))+intercept)

In [176]:
multi_step_window = WindowGenerator(
    input_width=200, label_width=1, shift=0,
    train_df=train_df, 
    val_df=val_df, 
    test_df=test_df,
    label_columns=['Discharge'],
    input_columns=input_columns)

# Multistep Model Runs

In [177]:
history = compile_and_fit(linear, multi_step_window)

val_performance['Multistep_Linear'] = linear.evaluate(multi_step_window.val)
performance['Multistep_Linear'] = linear.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1367/1367 [==============================] - 55s 40ms/step - loss: 0.1305 - mean_absolute_error: 0.2697 - val_loss: 0.7691 - val_mean_absolute_error: 0.7534
Epoch 2/20
1367/1367 [==============================] - 50s 36ms/step - loss: 0.1279 - mean_absolute_error: 0.2665 - val_loss: 0.6771 - val_mean_absolute_error: 0.7068
Epoch 3/20
1367/1367 [==============================] - 57s 42ms/step - loss: 0.1034 - mean_absolute_error: 0.2471 - val_loss: 0.6568 - val_mean_absolute_error: 0.6951
Epoch 4/20
1367/1367 [==============================] - 55s 40ms/step - loss: 0.1236 - mean_absolute_error: 0.2565 - val_loss: 0.5856 - val_mean_absolute_error: 0.6332
Epoch 5/20
1367/1367 [==============================] - 57s 41ms/step - loss: 0.1232 - mean_absolute_error: 0.2542 - val_loss: 0.7446 - val_mean_absolute_error: 0.7439
Epoch 6/20
386/386 [==============================] - 17s 28ms/step - loss: 1.0679 - mean_absolute_error: 0.9131


{'Linear': [0.40632280707359314, 0.5024384260177612],
 'Multistep_Linear': [0.6406988501548767, 0.6384437680244446]}

In [178]:
history = compile_and_fit(lstm_model, multi_step_window)

val_performance['Multistep_LSTM'] = lstm_model.evaluate(multi_step_window.val)
performance['Multistep_LSTM'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
performance

Epoch 1/20
1367/1367 [==============================] - 134s 97ms/step - loss: 0.0524 - mean_absolute_error: 0.1445 - val_loss: 1.2285 - val_mean_absolute_error: 0.8790
Epoch 2/20
1367/1367 [==============================] - 132s 97ms/step - loss: 0.0128 - mean_absolute_error: 0.0798 - val_loss: 1.1333 - val_mean_absolute_error: 0.8577
Epoch 3/20
1367/1367 [==============================] - 136s 99ms/step - loss: 0.0152 - mean_absolute_error: 0.0850 - val_loss: 1.0394 - val_mean_absolute_error: 0.8377
Epoch 4/20
1367/1367 [==============================] - 137s 100ms/step - loss: 0.0081 - mean_absolute_error: 0.0645 - val_loss: 1.0470 - val_mean_absolute_error: 0.8368
Epoch 5/20
1367/1367 [==============================] - 139s 102ms/step - loss: 0.0073 - mean_absolute_error: 0.0604 - val_loss: 0.8512 - val_mean_absolute_error: 0.7542
Epoch 6/20
1367/1367 [==============================] - 141s 103ms/step - loss: 0.0063 - mean_absolute_error: 0.0564 - val_loss: 0.9383 - val_mean_absolu

{'Linear': [0.40632280707359314, 0.5024384260177612],
 'Multistep_Linear': [0.6406988501548767, 0.6384437680244446],
 'Multistep_LSTM': [0.2811006009578705, 0.41266363859176636]}

In [179]:
# history = compile_and_fit(dnn_model, multi_step_window)

# val_performance['Multistep_DNN'] = dnn_model.evaluate(multi_step_window.val)
# performance['Multistep_DNN'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
# performance

# Retraining same model on different partitions of training, val, and test

### Can I create a for loop to do this?

In [180]:
n = 10000  #chunk row size
list_df = [df_all_chan[i:i+n] for i in range(0,df_all_chan.shape[0],n)]
list_df[0]

,0,1,2,3,4,5,6,7,8,9,...,2487,2488,2489,2490,2491,2492,2493,2494,2495,Discharge
0,1737.318436,1932.321258,1788.269101,3706.977556,1831.756626,1768.674227,1908.850695,3690.341997,1804.145150,2437.436803,...,3603.932847,2066.070800,2215.669390,1881.772352,3990.883595,1584.965970,1787.281257,1862.912440,4160.545583,9.947179
1,8161.349279,8245.783723,8118.189935,528.834231,468.890511,714.714310,728.051775,158.440806,82.719811,116.949415,...,8238.452283,8204.901437,8194.671705,8228.769841,8211.066134,8229.370787,8218.436955,8210.143402,8178.032001,9.934657
2,8242.942236,8205.640252,8093.980026,510.832905,451.884623,709.510655,709.932307,153.715844,83.404382,116.112236,...,8184.500407,8221.490571,8202.429647,8194.927561,8197.675324,8188.718827,8189.658071,8225.017259,8163.918322,9.929290
3,8159.780047,8196.654224,8113.873839,489.734947,438.657688,690.418208,701.896139,158.058233,83.003254,112.787627,...,8173.860868,8156.187127,8151.113841,8235.843141,8145.360321,8192.392266,8169.275389,8129.926491,8161.652382,9.923923
4,8198.498465,8161.470921,8120.452338,521.425978,444.272741,689.710217,695.164917,160.246640,84.054029,111.125592,...,8198.827286,8234.815465,8219.987333,8140.961605,8193.517740,8157.154485,8241.648596,8229.194191,8234.212679,9.918557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8216.136264,8341.752852,8240.940177,646.662313,521.615071,687.289448,496.409176,170.853606,118.160048,121.487678,...,8231.697103,8260.884095,8252.189323,8214.393657,8249.569663,8267.274630,8235.091427,8262.757726,8266.493848,5.450400
9996,8299.125644,8343.262260,8271.665761,633.745447,496.616703,671.251222,436.118079,180.486280,141.182780,109.466751,...,8257.331263,8243.016406,8315.547470,8330.524710,8278.497412,8282.190912,8288.338669,8231.309952,8300.629228,5.452200
9997,8226.304473,8211.995563,8214.226577,658.793542,490.893919,691.924094,489.356272,179.542433,169.462663,87.622215,...,8261.160411,8258.612412,8236.682803,8241.355549,8253.403740,8233.463012,8217.282709,8293.313696,8279.108940,5.454000
9998,8291.540311,8340.032374,8272.550011,685.838140,492.012415,669.942546,394.220818,207.499712,202.700682,101.572003,...,8227.621614,8260.764062,8322.770561,8263.860537,8309.207158,8301.337959,8313.773173,8264.281224,8241.038434,5.455800


In [181]:
val_performance={}
performance={}

In [ ]:
k = 1

for i in list_df:
    n = len(i)
    labels = list(i.index)
    
    df_all_chan_copy = df_all_chan.copy()
    
    train_df = df_all_chan_copy.drop(labels=labels, axis=0)
    
    val_df = i[int(n*0.0):int(n*0.6)]
    test_df = i[int(n*0.6):int(n*1.0)]
    
    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df = (train_df - train_mean) / train_std
    val_df = (val_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std
    
    history = compile_and_fit(dnn_model, multi_step_window)

    val_performance['Multistep_DNN_fold' + str(k)] = dnn_model.evaluate(multi_step_window.val)
    performance['Multistep_DNN_fold' + str(k)] = dnn_model.evaluate(multi_step_window.test, verbose=0)
    
    history = compile_and_fit(lstm_model, multi_step_window)

    val_performance['Multistep_LSTM_fold' + str(k)] = lstm_model.evaluate(multi_step_window.val)
    performance['Multistep_LSTM_fold' + str(k)] = lstm_model.evaluate(multi_step_window.test, verbose=0)
    
    k += 1
    
    

In [184]:
print(performance)

{'Multistep_DNN_fold1': [0.23176701366901398, 0.403778612613678], 'Multistep_LSTM_fold1': [0.2547881007194519, 0.40394505858421326], 'Multistep_DNN_fold2': [0.23435941338539124, 0.3971821963787079], 'Multistep_LSTM_fold2': [0.2535686790943146, 0.39431750774383545], 'Multistep_DNN_fold3': [0.20528961718082428, 0.37063658237457275], 'Multistep_LSTM_fold3': [0.23721162974834442, 0.3834927976131439], 'Multistep_DNN_fold4': [0.20735976099967957, 0.37081077694892883], 'Multistep_LSTM_fold4': [0.23417547345161438, 0.38287949562072754], 'Multistep_DNN_fold5': [0.22618788480758667, 0.394429087638855], 'Multistep_LSTM_fold5': [0.234232559800148, 0.3754889667034149], 'Multistep_DNN_fold6': [0.2064242959022522, 0.371305376291275], 'Multistep_LSTM_fold6': [0.2242651879787445, 0.3762040436267853], 'Multistep_DNN_fold7': [0.2180650383234024, 0.3826006054878235], 'Multistep_LSTM_fold7': [0.1998826265335083, 0.3568739593029022]}


In [153]:
# #Working in a weird way at the moment (chunks the data and runs on chunked data only, not the whole set each time)
# k = 1

# for i in list_df:
#     n = len(i)
    
#     train_df = i[int(n*0.0):int(n*0.7)]
#     val_df = i[int(n*0.7):int(n*0.9)]
#     test_df = i[int(n*0.9):int(n*1.0)]
    
#     train_mean = train_df.mean()
#     train_std = train_df.std()

#     train_df = (train_df - train_mean) / train_std
#     val_df = (val_df - train_mean) / train_std
#     test_df = (test_df - train_mean) / train_std
    
#     history = compile_and_fit(dnn_model, multi_step_window)

#     val_performance['Multistep_DNN_fold' + str(k)] = dnn_model.evaluate(multi_step_window.val)
#     performance['Multistep_DNN_fold' + str(k)] = dnn_model.evaluate(multi_step_window.test, verbose=0)
#     performance
    
#     k += 1
    

In [155]:
# print(performance)

In [154]:
#Obsolete way

# train_df = df_all_chan[int(n*0.0):int(n*0.7)]
# val_df = df_all_chan[int(n*0.8):int(n*1.0)]
# test_df = df_all_chan[int(n*0.7):int(n*0.8)]

# history = compile_and_fit(lstm_model, multi_step_window)

# val_performance['Multistep_LSTM_run2'] = lstm_model.evaluate(multi_step_window.val)
# performance['Multistep_LSTM_run2'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
# performance

# train_df = df_all_chan[int(n*0.3):int(n*1.0)]
# val_df = df_all_chan[int(n*0.1):int(n*0.3)]
# test_df = df_all_chan[int(n*0.0):int(n*0.1)]

# history = compile_and_fit(lstm_model, multi_step_window)

# val_performance['Multistep_LSTM_run3'] = lstm_model.evaluate(multi_step_window.val)
# performance['Multistep_LSTM_run3'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
# performance

# train_df = df_all_chan[int(n*0.0):int(n*0.7)]
# val_df = df_all_chan[int(n*0.8):int(n*1.0)]
# test_df = df_all_chan[int(n*0.7):int(n*0.8)]

# history = compile_and_fit(lstm_model, multi_step_window)

# val_performance['Multistep_LSTM_run4'] = lstm_model.evaluate(multi_step_window.val)
# performance['Multistep_LSTM_run4'] = lstm_model.evaluate(multi_step_window.test, verbose=0)
# performance

# train_df = df_all_chan[int(n*0.0):int(n*0.7)]
# val_df = df_all_chan[int(n*0.8):int(n*1.0)]
# test_df = df_all_chan[int(n*0.7):int(n*0.8)]

# history = compile_and_fit(dnn_model, multi_step_window)

# val_performance['Multistep_DNN_run2'] = dnn_model.evaluate(multi_step_window.val)
# performance['Multistep_DNN_run2'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
# performance

# train_df = df_all_chan[int(n*0.3):int(n*1.0)]
# val_df = df_all_chan[int(n*0.1):int(n*0.3)]
# test_df = df_all_chan[int(n*0.0):int(n*0.1)]

# history = compile_and_fit(dnn_model, multi_step_window)

# val_performance['Multistep_DNN_run3'] = dnn_model.evaluate(multi_step_window.val)
# performance['Multistep_DNN_run3'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
# performance

# train_df = df_all_chan[int(n*0.0):int(n*0.7)]
# val_df = df_all_chan[int(n*0.8):int(n*1.0)]
# test_df = df_all_chan[int(n*0.7):int(n*0.8)]

# history = compile_and_fit(dnn_model, multi_step_window)

# val_performance['Multistep_DNN_run4'] = dnn_model.evaluate(multi_step_window.val)
# performance['Multistep_DNN_run4'] = dnn_model.evaluate(multi_step_window.test, verbose=0)
# performance